In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt  # Library for visualisation
import numpy as np               # Numeric library for calculations
import pandas as pd              # Library for data analysis
from scipy import stats          # Library for advanced scientific calculations
import seaborn as sns            # Library for statistical visualization
import re

In [2]:
## reading the dataset
df_receipts = pd.read_excel("receipts.xlsx")
df_users = pd.read_excel("users.xlsx")
df_brands = pd.read_excel('brands.xlsx')

In [3]:
df_users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,$oid:5ff1e194b6a9d73a3a9f1052,1.0,$date:1609687444800,$date:1609687537858,consumer,Email,WI
1,$oid:5ff1e194b6a9d73a3a9f1052,1.0,$date:1609687444800,$date:1609687537858,consumer,Email,WI
2,$oid:5ff1e194b6a9d73a3a9f1052,1.0,$date:1609687444800,$date:1609687537858,consumer,Email,WI
3,$oid:5ff1e1eacfcf6c399c274ae6,1.0,$date:1609687530554,$date:1609687530597,consumer,Email,WI
4,$oid:5ff1e194b6a9d73a3a9f1052,1.0,$date:1609687444800,$date:1609687537858,consumer,Email,WI


In [4]:
df_users.shape

(495, 7)

In [5]:
##Checking for nulls
df_users.isnull().sum()

_id              0
active           1
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64

In [6]:
##percentage of nulls
null_percentage = df_users.isnull().mean()
for key, value in null_percentage.items():
    if value > 0:
        print(key, ':', value*100)

active : 0.20202020202020202
lastLogin : 12.525252525252526
signUpSource : 9.696969696969697
state : 11.313131313131313


In [7]:
##checking for duplicates in users dataset
df_users.duplicated().sum()

283

In [11]:
##Percentage of duplicates
df_users.duplicated().sum()/df_users.shape[0]*100

57.17171717171718

Data Quality issues in User
1. Out of 495 records, 283 records are duplicated i,e. 57.17% of the records in Users dataset are duplicates
2. There are 4 columns having null values. Since the percentage of null values are small, they can be ignored.


In [13]:
df_brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,$oid:601ac115be37ce2ead437551,511111019862,Baking,BAKING,$id:$oid:601ac114be37ce2ead437550|$ref:Cogs,test brand @1612366101024,NaN,NaN
1,$oid:601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,$id:$oid:5332f5fbe4b03c9a25efd0ba|$ref:Cogs,Starbucks,NaN,STARBUCKS
2,$oid:601ac142be37ce2ead43755d,511111819905,Baking,BAKING,$id:$oid:601ac142be37ce2ead437559|$ref:Cogs,test brand @1612366146176,NaN,TEST BRANDCODE @1612366146176
3,$oid:601ac142be37ce2ead43755a,511111519874,Baking,BAKING,$id:$oid:601ac142be37ce2ead437559|$ref:Cogs,test brand @1612366146051,NaN,TEST BRANDCODE @1612366146051
4,$oid:601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,$id:$oid:5332fa12e4b03c9a25efd1e7|$ref:Cogs,test brand @1612366146827,NaN,TEST BRANDCODE @1612366146827


In [14]:
df_brands.shape

(1167, 8)

In [15]:
##Checking for nulls
df_brands.isnull().sum()

_id                0
barcode            0
category         155
categoryCode     650
cpg                0
name               0
topBrand        1136
brandCode        269
dtype: int64

In [16]:
##percentage of nulls
null_percentage_brands = df_brands.isnull().mean()
for key, value in null_percentage_brands.items():
    if value > 0:
        print(key, ':', value*100)

category : 13.281919451585262
categoryCode : 55.69837189374465
topBrand : 97.34361610968296
brandCode : 23.050556983718938


In [18]:
df_brands.duplicated().sum()

0

Data quality issues in Brands:
1. 4 columns have null values. Out of the 4 columnns, TopBrand contains 97% null values, and CategoryCode contains 55% null values. Since 97% of the TopBrand values are null, the field is not adding any significance to the analysis and could be dropped. 

In [41]:
df_receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,$oid:5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",$date:1609687531000,$date:1609687531000,$date:1609687531000,$date:1609687536000,$date:1609687531000,500.0,$date:1609632000000,5.0,barcode:4011|description:ITEM NOT FOUND|finalP...,FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,$oid:5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",$date:1609687483000,$date:1609687483000,$date:1609687483000,$date:1609687488000,$date:1609687483000,150.0,$date:1609601083000,2.0,barcode:4011|description:ITEM NOT FOUND|finalP...,FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,$oid:5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,$date:1609687537000,$date:1609687537000,NaN,$date:1609687542000,NaN,5.0,$date:1609632000000,1.0,needsFetchReview:false|partnerItemId:1|prevent...,REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,$oid:5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,$date:1609687534000,$date:1609687534000,$date:1609687534000,$date:1609687539000,$date:1609687534000,5.0,$date:1609632000000,4.0,barcode:4011|description:ITEM NOT FOUND|finalP...,FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,$oid:5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,$date:1609687506000,$date:1609687506000,$date:1609687511000,$date:1609687511000,$date:1609687506000,5.0,$date:1609601106000,2.0,barcode:4011|description:ITEM NOT FOUND|finalP...,FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [43]:
df_receipts.shape

(1119, 15)

In [44]:
df_receipts.isnull().sum()

_id                          0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                   0
pointsAwardedDate          582
pointsEarned               510
purchaseDate               448
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus        17
totalSpent                 452
userId                      17
dtype: int64

In [45]:
##percentage of nulls
null_percentage_receipts = df_receipts.isnull().mean()
for key, value in null_percentage_receipts.items():
    if value > 0:
        print(key, ':', value*100)

bonusPointsEarned : 51.385165326184094
bonusPointsEarnedReason : 51.385165326184094
finishedDate : 49.240393208221626
pointsAwardedDate : 52.01072386058981
pointsEarned : 45.57640750670242
purchaseDate : 40.03574620196604
purchasedItemCount : 43.25290437890974
rewardsReceiptItemList : 39.32082216264522
rewardsReceiptStatus : 1.519213583556747
totalSpent : 40.39320822162645
userId : 1.519213583556747


In [46]:
df_receipts.duplicated().sum()

0

Data quality issues in Receipts
1. PointsEarned has 45% null values. The points were earned for some of the receipts but was not captured.
2. PurchasedItemCount: Determining whether customers who purchase several units of a product are eligible for bonuses  that necessitate the purchase of a specific number of brands will be complicated if there are a lot of missing values.